In [ ]:
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import pennylane as qml
import pickle
from tqdm import tqdm

# Define dataset path
dataset_path = "D:/datasets/PetImages"
categories = ["Cat", "Dog"]

# Check if dataset exists
if not os.path.exists(dataset_path):
    raise FileNotFoundError("Dataset directory not found!")

# Define Image Transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load Pretrained VGG16 Model
vgg16 = models.vgg16(pretrained=True)
model = nn.Sequential(*list(vgg16.children())[:-1])  # Remove final classifier
model.eval()

def load_images(category):
    category_path = os.path.join(dataset_path, category)
    if not os.path.exists(category_path):
        raise FileNotFoundError(f"{category} folder not found!")
    
    images = []
    for img_name in os.listdir(category_path)[:100]:  # Load limited images for testing
        img_path = os.path.join(category_path, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = transform(img)
            images.append(img)
    
    return torch.stack(images) if images else torch.empty(0)

def extract_features(images):
    if images.shape[0] == 0:
        return np.array([])
    with torch.no_grad():
        features = model(images).squeeze()
    return features.view(features.shape[0], -1).numpy()

def amplitude_encoding(features):
    feature_len = len(features[0])  # 25088 for VGG16
    num_qubits = int(np.ceil(np.log2(feature_len)))  # Next power of 2 (15 qubits for 32768)

    # Pad feature vector to match 2^num_qubits
    padded_len = 2 ** num_qubits
    padded_features = np.zeros((len(features), padded_len))
    padded_features[:, :feature_len] = features  # Fill original values

    dev = qml.device("default.qubit", wires=num_qubits)

    @qml.qnode(dev)
    def encode_feature_vector(feature_vector):
        qml.templates.AmplitudeEmbedding(feature_vector, wires=range(num_qubits), normalize=True)
        return [qml.expval(qml.PauliZ(i)) for i in range(num_qubits)]

    return np.array([encode_feature_vector(f) for f in padded_features])


# Process Data
encoded_data = {}
for category in categories:
    print(f"Processing {category} images...")
    images = load_images(category)
    features = extract_features(images)
    qfeatures = amplitude_encoding(features)
    encoded_data[category] = qfeatures

# Save Encoded Data
with open("quantum_encoded_dataf-VGG16.pkl", "wb") as f:
    pickle.dump(encoded_data, f)

print("Quantum encoded data saved successfully!")

In [2]:
import pickle

with open("quantum_encoded_dataf-VGG16.pkl", "rb") as f:
    data = pickle.load(f)

# Display contents head
for key, value in data.items():
    print(f"Key: {key}")

    if hasattr(value, 'shape'):
        print(f"  Shape: {value.shape}")
        print(f"  Sample Data: {value[:5]}")  

    else:
        print(f"  Value: {value}")

    print("\n" + "="*50 + "\n")


Key: Cat
  Shape: (100, 15)
  Sample Data: [[ 0.30744584  0.3140313  -0.01021218  0.00650647 -0.01859961  0.00208803
   0.03067205 -0.02548145  0.04621609  0.01535632 -0.01638981  0.04043715
   0.00401886  0.0125727   0.00378175]
 [ 0.38016246  0.15620541 -0.10914964 -0.21418308 -0.11442594  0.0019177
   0.09973423 -0.09932011 -0.17917055 -0.0273188  -0.02485525  0.02962766
   0.06984065 -0.01333068  0.00709992]
 [ 0.1827091   0.22371495 -0.09715515  0.03010768  0.01085691  0.04966486
   0.11637957  0.06302308 -0.14814628  0.02007559  0.0053967   0.00870771
   0.09983913 -0.02458192  0.00096232]
 [ 0.23736537  0.29812186 -0.02755987 -0.09567766  0.02821702  0.08294426
   0.0229663   0.06099487 -0.02896065  0.10892302 -0.15556594  0.03920613
   0.04725985  0.01876614 -0.00138254]
 [ 0.23591991  0.28449232 -0.10468675 -0.02277449  0.0235628   0.09927221
  -0.00257819  0.05346106 -0.03587455  0.0171419   0.06819305  0.00411254
   0.07810726  0.00429773  0.012983  ]]


Key: Dog
  Shape: (1